# Testing the non-Markovian Path Analisys Package

We are going to generate first a MC trajectory of a toy model 

In [1]:
import numpy as np

from NMpathAnalysis.nmtools.interval import Interval
from src.ensemble import *

In [2]:
'''
Function to mimic a MC simulation
'''
def mc_simulation(numsteps):
    x = 5
    I = Interval([0,100])
    mc_traj = []
    
    for i in range(numsteps):
        dx = random.uniform(-10,10)
        if (x + dx) in I:
            x = x + dx
        mc_traj.append(x)
    return np.array(mc_traj)

In [3]:
'''
Simple mapping funcion 
'''
def mapping_function(x):
    return int(x/10)

In [4]:
#Generating a short MC trajectoy
mc_trajectory = mc_simulation(10000)


## 1- Ensemble class (analysis of continuos trajectories)

Stores an esemble (array) of trajectories with extra functionalities. The ensemble could have a any number of trajectories including no trajectories at all.

### Transition probabilities

In [5]:
my_ensemble = Ensemble(mc_trajectory)

In [6]:
n_states = 10
C1 = my_ensemble._count_matrix(n_states, mapping_function)
print(C1)

[[ 623.  198.    0.    0.    0.    0.    0.    0.    0.    0.]
 [ 197.  439.  219.    0.    0.    0.    0.    0.    0.    0.]
 [   0.  218.  516.  258.    0.    0.    0.    0.    0.    0.]
 [   0.    0.  257.  572.  280.    0.    0.    0.    0.    0.]
 [   0.    0.    0.  279.  506.  251.    0.    0.    0.    0.]
 [   0.    0.    0.    0.  250.  480.  252.    0.    0.    0.]
 [   0.    0.    0.    0.    0.  252.  490.  262.    0.    0.]
 [   0.    0.    0.    0.    0.    0.  262.  509.  242.    0.]
 [   0.    0.    0.    0.    0.    0.    0.  242.  476.  267.]
 [   0.    0.    0.    0.    0.    0.    0.    0.  267.  935.]]


In [7]:
K1 = my_ensemble._mle_transition_matrix(n_states, mapping_function)
print(K1)

[[ 0.75883069  0.24116931  0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.23040936  0.51345029  0.25614035  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.21975806  0.52016129  0.26008065  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.23174031  0.51577998  0.25247971  0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.26930502  0.48841699  0.24227799
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.25458248  0.48879837
   0.25661914  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.25099602
   0.48804781  0.26095618  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.25863771  0.50246792  0.23889437  0.        ]
 [ 0.          0.          0.          0

### Defining states and computing MFPTs
The states are considered intervals in the is the class is Ensemble

In [8]:
stateA = [0,10]
stateB = [90,100]

my_ensemble.mfpts(stateA, stateB)

{'mfptAB': 299.6666666666667,
 'mfptBA': 337.6,
 'std_err_mfptAB': 76.553694542772703,
 'std_err_mfptBA': 66.343268937649825}

### Adding more trajectories to the ensemble or (ensemble + ensemble)

In [9]:
seq1 = mc_simulation(5000)
seq2 = mc_simulation(5000)
seq3 = mc_simulation(5000)

my_e1 = Ensemble(seq1)
my_e2 = Ensemble(seq2)
my_e3 = Ensemble(seq3)

ensemble1 = my_e1 + my_e2 + my_e3

In [10]:
stateA = [0,10]
stateB = [90,100]
ensemble1.mfpts(stateA,stateB)

{'mfptAB': 259.2,
 'mfptBA': 231.86666666666667,
 'std_err_mfptAB': 44.705465735932847,
 'std_err_mfptBA': 31.926225142535273}

In [11]:
e1 = Ensemble([1,2,3,4])
e2 = Ensemble([2,3,4,5])
e3 = Ensemble([2,1,1,4])

my_ensembles = [e1, e2, e3]

ensemble_tot = Ensemble([])

for traj in my_ensembles:
    ensemble_tot += traj

ensemble_tot.sequence
ensemble_tot.mfpts([1,1],[4,4])

{'mfptAB': 2.5,
 'mfptBA': 'NaN',
 'std_err_mfptAB': 0.35355339059327373,
 'std_err_mfptBA': 'NaN'}


## 2- DiscreteEnsemble class

We can generate a discrete trajectory from the same mapping function and we should obtain exaclty the same result:

In [12]:

d_ens = DiscreteEnsemble.from_ensemble(mc_trajectory,mapping_function)

print(d_ens.sequence[0][1:200])
d_ens.sequence.shape

[0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 1 1 2 1 2 1 1 1 0 0 0 0 0 0 0
 0 0 0 1 1 2 3 2 2 2 2 2 3 2 1 2 2 1 1 2 2 2 2 3 3 4 3 3 3 2 3 3 4 3 3 3 3
 4 5 4 3 3 3 3 3 2 2 2 3 4 5 5 5 4 4 4 3 3 3 3 3 2 2 3 3 3 3 3 3 2 1 1 0 0
 0 1 1 0 0 0 0 1 2 2 2 1 2 1 2 1 2 2 2 2 3 3 2 2 3 2 2 2 3 2 1 1 2 2 2 3 4
 4 5 6 5 4 5 5 5 5 4 3 3 4 5 5 6 6 6 6 7 7 7 7 7 7 7 7 8 9 8 9 9 9 9 9 8 9
 9 8 9 8 8 7 7 6 5 4 4 3 3 3]


(1, 10000)

In [13]:
C2 = d_ens._count_matrix(n_states)
print(C2)

[[ 623.  198.    0.    0.    0.    0.    0.    0.    0.    0.]
 [ 197.  439.  219.    0.    0.    0.    0.    0.    0.    0.]
 [   0.  218.  516.  258.    0.    0.    0.    0.    0.    0.]
 [   0.    0.  257.  572.  280.    0.    0.    0.    0.    0.]
 [   0.    0.    0.  279.  506.  251.    0.    0.    0.    0.]
 [   0.    0.    0.    0.  250.  480.  252.    0.    0.    0.]
 [   0.    0.    0.    0.    0.  252.  490.  262.    0.    0.]
 [   0.    0.    0.    0.    0.    0.  262.  509.  242.    0.]
 [   0.    0.    0.    0.    0.    0.    0.  242.  476.  267.]
 [   0.    0.    0.    0.    0.    0.    0.    0.  267.  935.]]


In [14]:
K2= d_ens._mle_transition_matrix(n_states)
print(K2)

[[ 0.75883069  0.24116931  0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.23040936  0.51345029  0.25614035  0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.21975806  0.52016129  0.26008065  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.23174031  0.51577998  0.25247971  0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.26930502  0.48841699  0.24227799
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.25458248  0.48879837
   0.25661914  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.25099602
   0.48804781  0.26095618  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.25863771  0.50246792  0.23889437  0.        ]
 [ 0.          0.          0.          0

### Defining states and computing MFPTs
The states are now considered sets, defining the states as follow we should obtain the same results

In [15]:
stateA = [0]
stateB = [9]

d_ens.mfpts(stateA, stateB)

{'mfptAB': 299.6666666666667,
 'mfptBA': 337.6,
 'std_err_mfptAB': 76.553694542772703,
 'std_err_mfptBA': 66.343268937649825}

The main function of this class would be to generate trajectories from the transition matrix

In [16]:
d_ens2 = DiscreteEnsemble.from_transition_matrix(K2, sim_length = 10000)

In [17]:
d_ens2.mfpts(stateA,stateB)

{'mfptAB': 146.3448275862069,
 'mfptBA': 203.39285714285714,
 'std_err_mfptAB': 24.823144998466823,
 'std_err_mfptBA': 28.605753456794378}